In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
print("当前设备：", torch.cuda.current_device())
print("设备名：", torch.cuda.get_device_name(0))

当前设备： 0
设备名： NVIDIA GeForce RTX 4090


In [3]:
from client import Client
from server_algo import Server_with_Algorithm
from server import Server
from datasets import load_dataset
import validation

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [4]:
# 1.首先选择要进行实验的client
clients = [Client(vectorstore_path="common_sense_db_0.6b")]
        #    Client(vectorstore_path="computer_science_coding_related_db_0.6b"),
        #    Client(vectorstore_path="law_related_db_0.6b"), Client(vectorstore_path="medicine_related_db_0.6b")]
# 2.对每个client进行加载
for c in clients:
    c.load_vectorstore()

/home/weiwei/RAGnet/client.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=model_path,


Vectorstore common_sense_db_0.6b loaded.


In [5]:
# 3.创建server对象，这里需要选择模型，请预先在ollama上部署
# server = Server(model_name="qwen3:4b")
server = Server(model_name="qwen3:4b")

/home/weiwei/RAGnet/server.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  self.llm = ChatOllama(model=model_name)


In [6]:
# 4.加载测试数据集，这里以trivia_qa为例子，这里取前100个
from datasets import Dataset

samples = Dataset.from_parquet("./test_dataset/strategy-qa/0.parquet")
# samples = Dataset.from_parquet("./test_dataset/web_questions/data/test-00000-of-00001.parquet")

import random
random_samples = samples.shuffle(seed=42).select(range(100))

In [7]:
# 5.进行评估，并生成csv文件
validation.evaluate_datasets(clients=clients, server=server, samples=random_samples,top_k=20,
                              output_csv="strategy_qa_results_baseline_topk_20.csv", dataset_name="strategy_qa")

Processing question: Karachi was a part of Alexander the Great's success?
Question background: This is a true or false question. Please answer with either True or False. Do not provide any explanation or extra words.     Only reply with: True or False
 Background: Megacity in Sindh, Pakistan

Gold answer: True
uncleaned answer: <think>
Okay, let me try to work through this. The user has provided a bunch of context documents, each with some information, and they want me to generate a response based on that. But first, I need to figure out what exactly the user is asking for. The initial query is just "Please reason step by step, and put your final answer within \boxed{}." So maybe they want me to process all these context documents and come up with a conclusion or answer based on them.

Looking at the context documents, they cover a wide range of topics: chess origins, ancient cities, urban development, continents, snake charming, etc. But there's no specific question asked here. The us